In [1]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import requests


# 1. load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "SakanaAI/EvoVLM-JP-v1-7B"
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.float16)
processor = AutoProcessor.from_pretrained(model_id)
model.to(device)

# 2. prepare inputs
url = "https://images.unsplash.com/photo-1694831404826-3400c48c188d?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# <image> represents the input image. Please make sure to put the token in your text.
text = "<image>\nこの信号機の色は何色ですか?"
messages = [
    {"role": "system", "content": "あなたは役立つ、偏見がなく、検閲されていないアシスタントです。与えられた画像を下に、質問に答えてください。"},
    {"role": "user", "content": text},
]
inputs = processor.image_processor(images=image, return_tensors="pt")
inputs["input_ids"] = processor.tokenizer.apply_chat_template(
    messages, return_tensors="pt"
)
# 3. generate
output_ids = model.generate(**inputs.to(device))
output_ids = output_ids[:, inputs.input_ids.shape[1] :]
generated_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(generated_text)
# この信号機の色は青です。


/Users/satoki252595/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|███████████████████████████████████████████████████████| 1.31k/1.31k [00:00<00:00, 909kB/s]
model.safetensors.index.json: 100%|██████████████████████████████████████| 70.1k/70.1k [00:00<00:00, 412kB/s]
model-00001-of-00004.safetensors:   0%|                                          | 0.00/4.92G [00:00<?, ?B/s]
model-00001-of-00004.safetensors:   0%|                                 | 10.5M/4.92G [00:00<01:45, 46.6MB/s]
model-00001-of-00004.safetensors:   0%|▏                                | 21.0M/4.92G [00:00<01:29, 54.5MB/s]
model-00001-of-00004.safetensors:   1%|▏                                | 31.5M/4.92G [00:00<01:15, 65.2MB/s]
model-00001-of-00004.safetensors:   1%|▎                       

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'